In [1]:
import pandas as pd
import sql_magic
import datetime
import folium
# pd.set_option('display.max_colwidth', -1)
%load_ext sql

In [2]:
%sql sqlite:////Users/mchen/Documents/Sumo/SQLite.db

'Connected: @/Users/mchen/Documents/Sumo/SQLite.db'

In [3]:
post_DB = pd.DataFrame.from_csv('''C:\\Users\\mchen\\Documents\\Sumo\\Jobs\\Map\\Criminal\\Australian_Post_Codes_Lat_Lon.csv''')
post_DB.to_sql('Australian_Post_Codes_Lat_Lon', con = 'sqlite:////Users/mchen/Documents/Sumo/SQLite.db')

post_DB = pd.DataFrame.from_csv('''C:\\Users\\mchen\\Documents\\Sumo\\Jobs\\Map\\Criminal\\Data_tables_Criminal_Incidents_Visualisation_year_ending_June_2019.csv''')
post_DB.to_sql('criminal', con = 'sqlite:////Users/mchen/Documents/Sumo/SQLite.db')

C:\Users\mchen\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.


ValueError: Table 'Australian_Post_Codes_Lat_Lon' already exists.

In [ ]:
%%sql
create table chriminal_latlon as 
select 
    a1.`Suburb/Town Name` as Suburb
    ,a1.`Postcode` as Postcode
    ,a1.`Incidents Recorded` as Incidents_cnt
    ,substr(a1.`Year ending June`, 1,4) as YYYY
    ,a2.lat
    ,a2.lon
from 
    criminal as a1
        left join Australian_Post_Codes_Lat_Lon as a2
            on a1.`Suburb/Town Name` = a2.suburb COLLATE NOCASE
            and a1.Postcode = a2.postcode COLLATE NOCASE
where
    a2.state = 'VIC'
limit 8
;

In [ ]:
%%sql
drop table chriminal_latlon_sum;
create table chriminal_latlon_sum as
select
    Suburb
    ,Postcode
    ,lat
    ,lon
    ,sum(Incidents_cnt) as Incidents_cnt
from 
    chriminal_latlon
where
    YYYY = '2019'
group by
    Suburb
    ,Postcode
    ,lat
    ,lon
;

In [4]:
%%sql rs <<
select *
from chriminal_latlon_sum
order by
    Incidents_cnt desc 

 * sqlite:////Users/mchen/Documents/Sumo/SQLite.db
Done.
Returning data to local variable rs


In [5]:
df = pd.DataFrame(rs)
df.columns = rs.field_names
df.head()

,Suburb,Postcode,lat,lon,Incidents_cnt
0,Melbourne,3000,-37.814563,144.970267,8109.0
1,Dandenong,3175,-37.987271,145.214317,6127.0
2,Frankston,3199,-38.145001,145.122477,5680.0
3,Mildura,3500,-34.181714,142.163072,4403.0
4,Shepparton,3630,-36.381171,145.399290,4361.0


In [6]:
m = folium.Map([-38.043995, 145.264296], zoom_start=5)

for index, row in df.iterrows():
    if row['Incidents_cnt'] >= 2000:
            marker_color = 'darkred'
            fill_color = 'darkred'
    elif row['Incidents_cnt'] >= 1000:
        marker_color = 'darkred'
        fill_color = 'red'
    elif row['Incidents_cnt'] >= 500:
        marker_color = 'darkred'
        fill_color = 'lightred'
    elif row['Incidents_cnt'] >= 100:
        marker_color = 'darkred'
        fill_color = 'orange'
    else:
        marker_color='darkred'
        fill_color = 'darkpurple'
    folium.CircleMarker(
              location=[row['lat'], row['lon']],
#               popup= str(row['Incidents_cnt']),
              tooltip= row['Suburb'] +': '+ str(int(row['Incidents_cnt'])),
              radius=row['Incidents_cnt']/100,
              color=marker_color,
              fill=True,
              fill_color=fill_color,
           ).add_to(m)
m

In [ ]:
m.save('C:\\Users\\mchen\\Documents\\Sumo\\Jobs\\Map\\chriminal.html')